Code to visualize the diminishing returns of tesselation. 

In [5]:
"""
Individual plot generation script for sphere tessellation analysis
Saves each of the 4 plots as separate SVG files in a predetermined directory
"""

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import os
import seaborn as sns

sns.set_theme()

# Create output directory
output_dir = Path("/home/markus/Pictures/tessellation_plots")
output_dir.mkdir(exist_ok=True)

# ---------- 1.  Theory helpers -------------------------------------------------
def sphere_tessellation_error(n_triangles: float, radius: float = 1.0,
                              C: float = 0.1) -> float:
    """
    Leading-order L2 surface-approximation error for a curved surface discretised
    with flat triangles of characteristic edge length h ∝ N^(-½).
    E(N) ∝ h² ∝ N^(-1).
    """
    h = 2 * np.pi / np.sqrt(n_triangles)            # h ∝ N^(-½)
    return C * h**2 * (4 * np.pi * radius**2)       # ⇢ E ∝ N^(-1)

# ---------- 2.  Discrete icosahedral levels ----------------------------------
levels          = np.arange(0, 8)                  # 0 … 7 subdivisions
triangles       = 20 * 4**levels                   # Nₗ = 20·4ˡ
errors          = sphere_tessellation_error(triangles)
improve_pct     = np.hstack(([np.nan],
                             (np.diff(errors) / errors[:-1]) * -100))
# Take absolute value of efficiency for log plotting
efficiency      = np.abs(np.hstack(([np.nan],
                                   np.diff(errors) / np.diff(triangles))))

# ---------- 3.  Continuous curves for smooth overlay --------------------------
N_smooth        = np.logspace(1, 6, 400)           # 10 … 1 000 000
error_smooth    = sphere_tessellation_error(N_smooth)
marginal_benef  = -np.gradient(error_smooth, N_smooth)  # dE/dN ∝ N^(-2)

# ---------- 4.  Save data for further use -------------------------------------
df = pd.DataFrame({
    "Level":      levels,
    "Triangles":  triangles,
    "Error":      errors,
    "Improve_%":  improve_pct,
    "Efficiency": efficiency
})
csv_path = output_dir / "sphere_tessellation_analysis.csv"
df.to_csv(csv_path, index=False)
print(f"Data table written to {csv_path.resolve()}")

# ---------- 5.  Individual Plot Generation ------------------------------------

# Plot 1: Error vs triangles (log–log)
fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(triangles, errors, "o", label="Icosahedral levels", markersize=8)
ax.loglog(N_smooth, error_smooth, "--", label="Theory  $E\\propto N^{-1}$", linewidth=2)
ax.set_xlabel("Number of triangles $N$", fontsize=12)
ax.set_ylabel("Approximation error $E(N)$", fontsize=12)
ax.set_title("Error vs Triangle Count", fontsize=14, fontweight='bold')
ax.grid(True, which="both", ls=":", alpha=0.7)
ax.legend(fontsize=11)
plt.tight_layout()
plt.savefig(output_dir / "plot1_error_vs_triangles.svg", format='svg', dpi=300)
plt.close()

# Plot 2: Improvement percentage
fig, ax = plt.subplots(figsize=(8, 6))
bars = ax.bar(levels[1:], improve_pct[1:], color="#4c72b0", alpha=0.8)
ax.set_xlabel("Tessellation level ℓ", fontsize=12)
ax.set_ylabel("Error reduction per level [%]", fontsize=12)
ax.set_title("Error Reduction per Subdivision Level\n≈ 75% drop each subdivision", fontsize=14, fontweight='bold')
ax.grid(axis="y", ls=":", alpha=0.7)
ax.set_xticks(levels[1:])
# Add value labels on bars
for bar, value in zip(bars, improve_pct[1:]):
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width()/2., height,
            f'{value:.1f}%', ha='center', va='bottom', fontsize=10)
plt.tight_layout()
plt.savefig(output_dir / "plot2_improvement_percentage.svg", format='svg', dpi=300)
plt.close()

# Plot 3: Efficiency (error drop per extra triangle)
fig, ax = plt.subplots(figsize=(8, 6))
ax.loglog(triangles[1:], efficiency[1:], "o-", color="#55a868", markersize=8, linewidth=2)
ax.set_xlabel("Number of triangles $N$", fontsize=12)
ax.set_ylabel("Efficiency |ΔE/ΔN|", fontsize=12)
ax.set_title("Efficiency: Error Reduction per Additional Triangle\nEfficiency falls as $N^{-2}$", fontsize=14, fontweight='bold')
ax.grid(True, which="both", ls=":", alpha=0.7)
plt.tight_layout()
plt.savefig(output_dir / "plot3_efficiency.svg", format='svg', dpi=300)
plt.close()

# Plot 4: Marginal benefit
fig, ax = plt.subplots(figsize=(8, 6))
ax.semilogx(N_smooth, marginal_benef, color="#c44e52", linewidth=2)
ax.set_xlabel("Number of triangles $N$", fontsize=12)
ax.set_ylabel("Marginal benefit $|dE/dN|$", fontsize=12)
ax.set_title("Marginal Benefit: Rate of Error Reduction\nMarginal benefit ∝ $N^{-2}$", fontsize=14, fontweight='bold')
ax.grid(True, which="both", ls=":", alpha=0.7)
plt.tight_layout()
plt.savefig(output_dir / "plot4_marginal_benefit.svg", format='svg', dpi=300)
plt.close()

print(f"\nAll plots saved as SVG files in: {output_dir.resolve()}")
print("Generated files:")
for svg_file in sorted(output_dir.glob("*.svg")):
    print(f"  - {svg_file.name}")


Data table written to /home/markus/Pictures/tessellation_plots/sphere_tessellation_analysis.csv

All plots saved as SVG files in: /home/markus/Pictures/tessellation_plots
Generated files:
  - plot1_error_vs_triangles.svg
  - plot2_improvement_percentage.svg
  - plot3_efficiency.svg
  - plot4_marginal_benefit.svg
